## Импорты

In [4]:
# !pip3 install matplotlib tqdm numpy torch

In [8]:
# %matplotlib inline
# import matplotlib.pyplot as plt

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader

import os
from tqdm import tqdm_notebook, trange, tqdm
import pickle
import numpy as np
from IPython.display import clear_output
import jedi
import tokenize
from io import BytesIO

In [2]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [3]:
# device

device(type='cpu')

# Playing with jedi module

In [3]:
code = '''from __future__ import print_function
from socket import timeout
import os
import sys
import codecs
import re
import markdown
try:
    from urllib2 import urlopen
    from urllib2 import HTTPError
    from urllib2 import URLError
except ImportError:
    from urllib.request import urlopen
    from urllib.error import HTTPError
    from urllib.error import URLError


def function874(arg89):
    var4252 = False
    try:
        var4262 = urlopen(arg89, timeout=2)
        var4252 = var4262.code == 200
    except HTTPError as var3998:
        print(var3998, file=sys.stderr)
    except URLError as var3998:
        print(var3998, file=sys.stderr)
    except timeout as var3998:
        print(var3998, file=sys.stderr)
    except Exception as var3998:
        print(var3998, file=sys.stderr)
    return v'''

script = jedi.Script(code)#source, 3, len('datetime.da'), 'example.py')
script.completions()


[<Completion: var3998>,
 <Completion: var4252>,
 <Completion: var4262>,
 <Completion: vars>]

In [4]:
code = '''
def foo(i):
    f'''

script = jedi.Script(code)#source, 3, len('datetime.da'), 'example.py')
script.completions()


[<Completion: filter>,
 <Completion: float>,
 <Completion: foo>,
 <Completion: for>,
 <Completion: format>,
 <Completion: from>,
 <Completion: frozenset>]


* суммирую все эмбеддинги, которые встретил до текущего ключевого слова, конкатенирую к эмбеддингу текущего слова
    - итерируюсь по файлам из train_files.txt
    - иду с помощью tokenize + isidentifier(), выбираю из кода identifiers, которые есть в mapping.map
        - может ли регулярка возвращать индекс начала распознанного слова?
        - сохраняю в массив `[[identifier, index], [...], ...]`
    - предсохраняю номера токенов, которые войдут в сумму
    - подаю в jedi предыдущий код + первую букву identifier'а (индекс от регулярки + 1)
    - итерируюсь по предсказаниям и добавляю объекты в train такого вида:
    ```
    [
        arr_of_prev_tokens,
        predicted_token (число из mapping.map),
        real_token (число из mapping.map)
    ]```
    - дамплю в пикл `{file_name: train}`
* подаю на вход полносвязному слою
* беру сигмоиду
* подаю в кросс-энтропию

In [10]:
def prepare_samples(path, samples_path):
    def get_filenames(path):
        with open(path) as f:
            filenames = f.read().split('\n')
        return filenames
    
    def load_mapping(path):
        with open(path, 'rb') as f:
#         print(base_path + '/mapping.map')
            word_to_id = pickle.load(f)
        return word_to_id

    def predict_tokens(code):
        script = jedi.Script(code)
        return script.completions()
    
    def load_preprocessed(path):
        try:
            with open(path, 'rb') as f:
                samples = pickle.load(f)
        except FileNotFoundError:
            samples = {}
        return samples
    
    def save_preprocessed(samples_dict_path, samples_dict):
        with open(samples_dict_path, 'wb') as f:
            samples = pickle.dump(samples_dict, f)
    
    filenames = get_filenames(path + samples_path)
    word_to_id = load_mapping('./mapping.map')
    
    samples_dict_path = './ranking_samples_' + samples_path[:3]
    samples_dict = load_preprocessed(samples_dict_path)

    for i, filename in enumerate(tqdm_notebook(filenames)):
        if filename in samples_dict:
            continue

        with open(path + filename) as f:
            code = f.read()
        
        samples = []
        prev_tokens = []
        g = tokenize.tokenize(BytesIO(code.encode("utf-8")).readline)
        code_lines = code.split('\n')
        for toktype, tokval, start, _, _ in g:
            if toktype == tokenize.NAME and tokval in word_to_id:
                row, col = start
#                     print(word_to_id[tokval])
                real_token = word_to_id[tokval]

                # predict using jedi
#                     print(row, col, code_lines[row-1][:col+1])
                prev_code = '\n'.join(code_lines[:row-1]) + '\n' + code_lines[row-1][:col+1]
#                     print(prev_code)
                preds = predict_tokens(prev_code)
                for pred in preds:
                    completion = pred.full_name
                    if completion in word_to_id:
                        completion_id = word_to_id[completion]
                        samples.append([
                            prev_tokens[:],
                            completion_id,
                            real_token
                        ])
                prev_tokens.append(real_token)
#                     break
        samples_dict[filename] = samples
        save_preprocessed(samples_dict_path, samples_dict)
#         break
    save_preprocessed(samples_dict_path, samples_dict)
    return samples_dict

path = './pycodesuggest_py_repos_normalised/'
samples_path = 'train_files.txt'
samples = prepare_samples(path, samples_path)
        

InvalidPythonEnvironment: Could not get version information for '/usr/local/opt/python/bin/python3.7': BlockingIOError(35, 'Resource temporarily unavailable')

In [15]:
import tokenize
from io import BytesIO
from keyword import iskeyword

def load_code(path):
    pass

def process_code(code):
#     s = code #
    s = "def twoπ(a,b):"
    g = tokenize.tokenize(BytesIO(s.encode("utf-8")).readline)
    new_s = ''
    for toktype, tokval, st, end, _ in g:
        if toktype == tokenize.NAME:
            if iskeyword(tokval):
                print("KEYWORD", tokval, tokval.isidentifier(), st)
            else:
                print('NAME', tokval, tokval.isidentifier(), st)
        else:
            print(toktype, tokval)
        new_s += f'{tokval} '
    print(new_s)

process_code(None)

57 utf-8
KEYWORD def True (1, 0)
NAME twoπ True (1, 4)
53 (
NAME a True (1, 9)
53 ,
NAME b True (1, 11)
53 )
53 :
4 
0 
utf-8 def twoπ ( a , b ) :   


In [35]:
print(new_s[8:].strip())

def foo ( i ) : 
      f


In [36]:
script = jedi.Script(new_s[8:].strip())
script.completions()

[<Completion: filter>,
 <Completion: float>,
 <Completion: foo>,
 <Completion: for>,
 <Completion: format>,
 <Completion: from>,
 <Completion: frozenset>]